<h1><center>Training</center></h1>


Architecture and loss functions inspired and adapted from https://github.com/ternaus/kaggle_dstl_submission

In [1]:
import os

import numpy as np
import keras.backend as K
from keras.layers import merge, Input, Conv2D, MaxPooling2D, UpSampling2D, Cropping2D, Flatten, Dense, BatchNormalization, Dropout
from keras.models import Model
from keras.layers.merge import concatenate
from keras.optimizers import Adam
from keras.backend import binary_crossentropy
from keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from PIL import Image

from generator import myGenerator

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# set up the gpu id
os.environ['CUDA_VISIBLE_DEVICES'] = "1"

The loss function used is a sum of binary crossentropy and jaccard index as defined below.

In [3]:
smooth = 1e-12

def jaccard_coef(y_true, y_pred):
    intersection = K.sum(y_true * y_pred, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)

def jaccard_coef_int(y_true, y_pred):
    y_pred_pos = K.round(K.clip(y_pred, 0, 1))

    intersection = K.sum(y_true * y_pred_pos, axis=[0, -1, -2])
    sum_ = K.sum(y_true + y_pred_pos, axis=[0, -1, -2])

    jac = (intersection + smooth) / (sum_ - intersection + smooth)

    return K.mean(jac)


def jaccard_coef_loss(y_true, y_pred):
    return -K.log(jaccard_coef(y_true, y_pred)) + binary_crossentropy(y_pred, y_true)

Architecture inspired from the U-Net paper (https://arxiv.org/pdf/1505.04597.pdf) 

In [4]:
def get_unet(num_channels, img_rows, img_cols):
    
    inputs = Input((img_rows, img_cols, num_channels))
    conv1 = Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(inputs)
    conv1 = BatchNormalization()(conv1)
    conv1 = Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = Conv2D(64, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(pool1)
    conv2 = BatchNormalization()(conv2)
    conv2 = Conv2D(64, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(128, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(pool2)
    conv3 = BatchNormalization()(conv3)
    conv3 = Conv2D(128, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)

    conv4 = Conv2D(256, 3, padding='same', kernel_initializer ='he_uniform', activation = 'elu')(pool3)
    conv4 = BatchNormalization()(conv4)
    conv4 = Conv2D(256, 3, padding='same', kernel_initializer ='he_uniform', activation = 'elu')(conv4)
    conv4 = BatchNormalization()(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(conv4)
    
    conv5 = Conv2D(512, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(pool4)
    conv5 = BatchNormalization()(conv5)
    conv5 = Conv2D(512, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv5)
    conv5 = BatchNormalization()(conv5)

    up6 = merge([UpSampling2D(size=(2, 2))(conv5), conv4], mode='concat', concat_axis=3)
    conv6 = Conv2D(256, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(up6)
    conv6 = BatchNormalization()(conv6)
    conv6 = Conv2D(256, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv6)
    conv6 = BatchNormalization()(conv6)

    up7 = merge([UpSampling2D(size=(2, 2))(conv6), conv3], mode='concat', concat_axis=3)
    conv7 = Conv2D(128, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(up7)
    conv7 = BatchNormalization()(conv7)
    conv7 = Conv2D(128, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv7)
    conv7 = BatchNormalization()(conv7)

    up8 = merge([UpSampling2D(size=(2, 2))(conv7), conv2], mode='concat', concat_axis=3)
    conv8 = Conv2D(64, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(up8)
    conv8 = BatchNormalization()(conv8)
    conv8 = Conv2D(64, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv8)
    conv8 = BatchNormalization()(conv8)

    up9 = merge([UpSampling2D(size=(2, 2))(conv8), conv1], mode='concat', concat_axis=3)
    conv9 = Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(up9)
    conv9 = BatchNormalization()(conv9)
    conv9 = Conv2D(32, 3, padding='same', kernel_initializer='he_uniform', activation = 'elu')(conv9)
    crop9 = Cropping2D(cropping=((16, 16), (16, 16)))(conv9)
    conv9 = BatchNormalization()(conv9)
    conv10 = Conv2D(1, 1, activation='sigmoid')(conv9)

    model = Model(inputs=[inputs], outputs=[conv10])
    adam = Adam(lr=1e-3)
    model.compile(adam, loss=jaccard_coef_loss, metrics=['binary_crossentropy', jaccard_coef_int])
    return model

For training, we random sample tiles of 256x256 from the training images.

In [7]:
img_size = 256
batch_size = 16
train_steps = 10000//batch_size
val_steps = 1000//batch_size

In [8]:
model = get_unet(3, img_size, img_size)

/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:33: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/keras/legacy/layers.py:465: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  name=name)
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:39: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:45: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py:51: UserWarning: The `m

Data augmentation

In [9]:
def flip_axis(x, axis):
    x = np.asarray(x).swapaxes(axis, 0)
    x = x[::-1, ...]
    x = x.swapaxes(0, axis)
    return x

In [10]:
# change the path to your data folders 
train = [os.path.join('/root/data/hackathon/building_massa_dataset/images/train/', f ) for f in os.listdir('/root/data/hackathon/building_massa_dataset/images/train/') if f.endswith('.tiff')]
val = [os.path.join('/root/data/hackathon/building_massa_dataset/images/valid/', f ) for f in os.listdir('/root/data/hackathon/building_massa_dataset/images/valid/') if f.endswith('.tiff')]

In [11]:
# create the generators
train_generator = myGenerator(train, train_steps, batch_size, (img_size, img_size, 3))
validation_generator = myGenerator(val, val_steps, batch_size, (img_size, img_size, 3))

In [12]:
reduce_lr = ReduceLROnPlateau(monitor='val_jaccard_coef_int',
                              factor=0.5, 
                              patience=1, 
                              min_lr=1e-6)
checkpoint = ModelCheckpoint('/root/data/hackathon/thomas_augmentation_weights_{epoch:02d}.hdf5')

In [13]:
callbacks_list = [reduce_lr, checkpoint]

Model should converge relatively fast. One epoch takes about 10 min on a single GTX TITAN

In [ ]:
# start training
history = model.fit_generator(
        generator=train_generator,
        steps_per_epoch=train_steps,
        epochs=50,
        verbose=1,
        validation_data=validation_generator,
        validation_steps=val_steps,
        callbacks=callbacks_list)

Epoch 1/50
625/625 [==============================] - 557s 892ms/step - loss: 3.1881 - binary_crossentropy: 0.9919 - jaccard_coef_int: 0.3455 - val_loss: 2.4198 - val_binary_crossentropy: 0.7634 - val_jaccard_coef_int: 0.3477
Epoch 2/50
625/625 [==============================] - 563s 901ms/step - loss: 2.3026 - binary_crossentropy: 0.9121 - jaccard_coef_int: 0.4437 - val_loss: 1.9446 - val_binary_crossentropy: 0.7331 - val_jaccard_coef_int: 0.4581
Epoch 3/50
301/625 [=============>................] - ETA: 4:33 - loss: 2.1215 - binary_crossentropy: 0.8946 - jaccard_coef_int: 0.4773